In [ ]:
import pandas as pd
import os

def get_data(data_file, columns_file):
    input_path = os.path.join(os.getcwd(), "datasets")
    columns = pd.read_csv(os.path.join(input_path, columns_file), sep="\t", header=None)
    columns = columns[0].str.partition(" (from ")[0].tolist()
    columns.insert(0, "Name")
    return pd.read_csv(os.path.join(input_path, data_file), sep="\t", names=columns)

data_file = "Untitled spreadsheet - Sheet1.tsv"
columns_file = "Untitled spreadsheet - Sheet2.tsv"
df = get_data(data_file, columns_file)

data_file = "Untitled spreadsheet - Sheet3.tsv"
columns_file = "Untitled spreadsheet - Sheet4.tsv"
df0 = get_data(data_file, columns_file)

def update_df(df, df0):
    rows = []
    for r in range(len(df0)):
        if df0.iloc[df0.last_valid_index()-r].Email == df.iloc[df.last_valid_index()].Email:
            return pd.concat([df,pd.DataFrame(rows)],ignore_index=True)
        else:
            rows.insert(0,pd.Series(df0.iloc[df0.last_valid_index()-r],index=df.columns))
            rows[0].Run = "2023 podzim"
    return pd.concat([df,pd.DataFrame(rows)],ignore_index=True)

df = update_df(df, df0)

In [51]:
report = pd.Series(dtype=int)
report['Applications'] = (len(df))
report['Participants'] = (len(df[df['Participation'] > 0]))
report['Dropout'] = (report['Applications'] - report['Participants']) / report['Applications']

report


Applications    266.000000
Participants    171.000000
Dropout          35.714286
dtype: float64

In [43]:
report.Applications.astype(int)

266.0

In [ ]:
report.Applications = report.Applications.astype(int)

In [ ]:
    
    {
    "Applications": len(df),
    "Participants": len(df.Participation[df.Participation > 0]),
    "Dropout": len(df.Participation[df.Participation > 0])/len(df),
    },dtype=int)
print(report)

In [ ]:
report.Applications.astype(int)